In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from scipy.special import logit, expit
import pickle
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

In [10]:
glist = ['air_x68', 'airx_w70', 'airx_x70', 'cbam_w69', 'cbam_x69',
           'preresnet_x67',
           'resnet_x80', 'se_resnext_w29'
           ]

In [3]:
nfold = 4

oof_dir = '../../Human_Protein_Atlas/metafeatures/oof/'

In [4]:

PATH_TO_TRAIN = '../../Human_Protein_Atlas/input/train/'
PATH_TO_TEST = '../../Human_Protein_Atlas/input/test/'
PATH_TO_TARGET = '../../Human_Protein_Atlas/input/train.csv'
PATH_TO_SUB = '../../Human_Protein_Atlas/input/sample_submission.csv'

In [5]:
sub = pd.read_csv(PATH_TO_SUB)

In [13]:
df = pd.read_csv(PATH_TO_TARGET)
df.set_index('Id',inplace=True)
print(df.head())
print(df.shape)

                                       Target
Id                                           
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18
(31072, 1)


In [14]:
file_list = list(df.index.values)

ss = pd.read_csv(PATH_TO_SUB)
ss.set_index('Id',inplace=True)
print(ss.head())
print(ss.shape)

                                      Predicted
Id                                             
00008af0-bad0-11e8-b2b8-ac1f6b6435d0          0
0000a892-bacf-11e8-b2b8-ac1f6b6435d0          0
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0          0
0008baca-bad7-11e8-b2b9-ac1f6b6435d0          0
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0          0
(11702, 1)


In [15]:
oof = df.copy()
# print(oof.head())
# print(oof.shape)
eps = 1e-6

In [16]:
oof = df.copy()
# print(oof.head())
# print(oof.shape)
eps = 1e-6

for k,m in enumerate(glist):
    # print(m),
    first = True
    for fold in range(nfold):
        idf, pooff, yooff = pickle.load(open(oof_dir+m+'_'+str(fold)+'.pkl','rb'))
        # change to probability scale
        pooff = expit(pooff)
        

        if first:
            id = idf
            poof = pooff.copy()
            yoof = yooff.copy()
            first = False
        else:
            id = id + idf
            poof = np.concatenate((poof,pooff))
            yoof = np.concatenate((yoof,yooff))
    
    mdf = pd.DataFrame({'Id':id})
    # print(poof.shape)
    if k==0: mdf['y'] = [yoof[i] for i in range(yoof.shape[0])]
    mdf[m] = [poof[i] for i in range(poof.shape[0])]
    mdf.set_index('Id',inplace=True)
    oof = oof.join(mdf)

In [17]:
oof.shape

(31072, 10)

In [18]:
oof.head()

,Target,y,air_x68,airx_w70,airx_x70,cbam_w69,cbam_x69,preresnet_x67,resnet_x80,se_resnext_w29
Id,,,,,,,,,,
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.9787988]], [[0.0052320026]], [[0.01596593...","[[[0.99222034]], [[0.0460565]], [[0.42665052]]...","[[[0.9887283]], [[0.0044118366]], [[0.0483257]...","[[[0.9861458]], [[0.027592985]], [[0.02133973]...","[[[0.988283]], [[0.0025513505]], [[0.004176334...","[[[0.8970872]], [[0.008869448]], [[0.055458836...","[[[0.9969104]], [[0.00054191396]], [[0.0029697...","[[[0.97751975]], [[0.0056676073]], [[0.0086767..."
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[[[0.9482032]], [[0.99923646]], [[0.2553666]],...","[[[0.98804307]], [[0.99944]], [[0.8884882]], [...","[[[0.9356185]], [[0.9996531]], [[0.28268027]],...","[[[0.8147194]], [[0.9847713]], [[0.8856964]], ...","[[[0.89690334]], [[0.9982436]], [[0.50983506]]...","[[[0.7879986]], [[0.99876785]], [[0.20188548]]...","[[[0.95630693]], [[0.98895735]], [[0.4966281]]...","[[[0.8519948]], [[0.99280554]], [[0.4493405]],..."
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[[[0.97671866]], [[0.008819221]], [[0.26768178...","[[[0.9762942]], [[0.00044200078]], [[0.1994801...","[[[0.95093626]], [[0.002990259]], [[0.35776192...","[[[0.906691]], [[0.0030315567]], [[0.18325233]...","[[[0.9529328]], [[0.0063327136]], [[0.38913277...","[[[0.9673825]], [[0.014083675]], [[0.49590272]...","[[[0.9538216]], [[0.028962485]], [[0.6092043]]...","[[[0.97027844]], [[0.0054005343]], [[0.0740023..."
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.75368506]], [[0.9990901]], [[0.007122622]...","[[[0.8679903]], [[0.9959116]], [[0.006667033]]...","[[[0.89748365]], [[0.9986174]], [[0.006502283]...","[[[0.97068834]], [[0.99854183]], [[0.016839104...","[[[0.94940335]], [[0.9986092]], [[0.0075771045...","[[[0.8388359]], [[0.99796]], [[0.021880563]], ...","[[[0.90697163]], [[0.9907246]], [[0.03382511]]...","[[[0.7687788]], [[0.9998683]], [[0.04143162]],..."
001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.6745915]], [[0.0005763447]], [[0.01309148...","[[[0.18778084]], [[0.0002924658]], [[0.0095870...","[[[0.0859067]], [[0.00020192834]], [[0.0059314...","[[[0.040219273]], [[0.0010586925]], [[0.066332...","[[[0.030123226]], [[0.000552267]], [[0.0271959...","[[[0.2969415]], [[0.0012038597]], [[0.02706289...","[[[0.754145]], [[0.004845952]], [[0.008448062]...","[[[0.1935358]], [[0.003470425]], [[0.09278208]..."


In [20]:
y = np.array(oof.y.values.tolist())
ymean = np.mean(y,axis=0)
print(ymean)

[4.1468203e-01 4.0357880e-02 1.1653579e-01 5.0238155e-02 5.9796602e-02
 8.0876671e-02 3.2440782e-02 9.0821318e-02 1.7057158e-03 1.4482492e-03
 9.0113288e-04 3.5176363e-02 2.2142122e-02 1.7282441e-02 3.4307417e-02
 6.7584962e-04 1.7057158e-02 6.7584962e-03 2.9029351e-02 4.7695674e-02
 5.5355304e-03 1.2155639e-01 2.5811020e-02 9.5423535e-02 1.0363027e-02
 2.6480433e-01 1.0556128e-02 3.5401649e-04]


In [21]:
thresholds =np.transpose(np.array([list(np.linspace(0.01,0.99, 99))]*28))

In [22]:
air_x68 = np.array(oof.air_x68.values.tolist())
air_x68 = air_x68.reshape((31072, 28))
Big_Tensor = (air_x68[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7191127624933157

In [23]:
airx_w70 = np.array(oof.airx_w70.values.tolist())
airx_w70 = airx_w70.reshape((31072, 28))
Big_Tensor = (airx_w70[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7270174019390939

In [24]:
airx_x70 = np.array(oof.airx_x70.values.tolist())
airx_x70 = airx_x70.reshape((31072, 28))
Big_Tensor = (airx_x70[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7441451689139543

In [25]:
cbam_w69 = np.array(oof.cbam_w69.values.tolist())
cbam_w69 = cbam_w69.reshape((31072, 28))
Big_Tensor = (cbam_w69[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7070331096093305

In [26]:
cbam_x69 = np.array(oof.cbam_x69.values.tolist())
cbam_x69 = cbam_x69.reshape((31072, 28))
Big_Tensor = (cbam_x69[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7334733436769992

In [27]:
preresnet_x67 = np.array(oof.preresnet_x67.values.tolist())
preresnet_x67 = preresnet_x67.reshape((31072, 28))
Big_Tensor = (preresnet_x67[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7210377682235267

In [28]:
resnet_x80 = np.array(oof.resnet_x80.values.tolist())
resnet_x80 = resnet_x80.reshape((31072, 28))
Big_Tensor = (resnet_x80[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.6646229786849619

In [29]:
se_resnext_w29 = np.array(oof.se_resnext_w29.values.tolist())
se_resnext_w29 = se_resnext_w29.reshape((31072, 28))
Big_Tensor = (se_resnext_w29[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.6840275346144953

In [30]:
thresholds_0 = np.linspace(0.01, 0.99, 99)
train_eight = np.hstack((air_x68, airx_w70, airx_x70, cbam_w69, cbam_x69, preresnet_x67, resnet_x80, se_resnext_w29))
train_eight.shape

(31072, 224)

In [31]:
cluster4x_folds = pd.read_csv('../Russ/cluster4x_folds.csv')

In [32]:
oof_folds = oof.merge(cluster4x_folds, on='Id')

In [33]:
cluster4 = oof_folds.cluster4.values

In [37]:
all_columns = set(range(224))

all_good_features = []
all_best_scores = []

for ii in tqdm(range(28)):
    print("Fitting feature", ii)
    first_eight =  set(np.array(range(8))*(28)+ii)
    the_rest = np.array(list(all_columns - first_eight))
    the_list = list(np.sort(list(first_eight)))+list(the_rest)
    best_score = 0
    good_features = []
    for feature in the_list:
        new_oof = np.zeros(train_eight.shape[0])
        temp_good_features = good_features+[feature]
        for i in range(4):
            val_idx = np.where(cluster4 == i)[0]
            trn_idx = np.where(cluster4 != i)[0]
            train_x, val_x = train_eight[:,temp_good_features][trn_idx, :], train_eight[:,temp_good_features][val_idx, :]
            train_y, val_y = y[:,ii][trn_idx], y[:,ii][val_idx]
            clf = Ridge(alpha=0)
            clf.fit(train_x, train_y)
            oof_preds = clf.predict(val_x)
            new_oof[val_idx] = oof_preds
            
        scaler = MinMaxScaler()
        new_oof = scaler.fit_transform(new_oof.reshape(-1,1))
        scores = [f1_score(y[:,ii], (new_oof > th)*1.,) for th in thresholds_0]
        new_score = np.max(scores)
        if new_score > best_score:
            print(feature)
            print(new_score)
            best_score = new_score
            good_features.append(feature)
            
    all_best_scores.append(best_score)
            
    all_good_features.append(good_features)
        

  0%|          | 0/28 [00:00<?, ?it/s]

Fitting feature 0
0
0.8397904981241411
28
0.845354006686997
56
0.8479749851101845
84
0.8514441899977839
112
0.8521933085501859
196
0.852635458901585
12
0.852789256198347
13
0.8528581380852858
16
0.8530544854155202
33
0.8531967030519045
36
0.8532709586396376
38
0.8532808496416502
42
0.8533224380641088
126
0.8534085845833952
162
0.8534105247532101
167
0.8534313179675611
180
0.8535126499981426
202
0.8535779271417431
206
0.8536096256684491


  4%|▎         | 1/28 [03:20<1:30:21, 200.80s/it]

Fitting feature 1
1
0.8453768453768453
29
0.8611111111111112
57
0.8641878669275929
85
0.869737887212073
197
0.8712091374556913
16
0.8712871287128713
17
0.871310507674144
19
0.8716323296354992
21
0.87199684915321
23
0.8723404255319149
35
0.872684272763106
41
0.8730283911671923
60
0.8733727810650886


  7%|▋         | 2/28 [06:07<1:22:38, 190.70s/it]

...

 93%|█████████▎| 26/28 [1:14:48<05:55, 177.66s/it]

Fitting feature 26
26
0.5256410256410255
54
0.6252045826513911
4
0.6262295081967214
17
0.6272577996715927
19
0.6282894736842105
28
0.6293245469522241
84
0.6303630363036303


 96%|█████████▋| 27/28 [1:17:27<02:52, 172.11s/it]

Fitting feature 27
27
0.7272727272727273
55
0.7692307692307692
139
0.8


100%|██████████| 28/28 [1:20:01<00:00, 166.79s/it]


In [38]:
np.mean(all_best_scores)

0.7716383951904737

In [39]:
np.save('all_good_features_4', all_good_features)

In [40]:
sub_dir = '../../Human_Protein_Atlas/metafeatures/sub/'

In [41]:
folds = []

for fold in range(nfold):
    preds = []
    for k,m in enumerate(glist):
        idf, pooff, yooff = pickle.load(open(sub_dir+m+'_'+str(fold)+'_mm.pkl','rb'))
        pooff = pooff.reshape((11702, 28))
        preds.append(pooff)
    folds.append(preds)

In [43]:
len(folds)

4

In [44]:
best_scores = []
best_thresholds = []
total_preds = []
total_preds_all_folds = []
oofs = []
for ii in tqdm(range(28)):
    print("Fitting feature", ii)
    new_oof = np.zeros(train_eight.shape[0])
    new_oof = new_oof.reshape(-1,1)
    feature_preds = 0
    feature_preds_list = []
    for i in range(4):
        val_idx = np.where(cluster4 == i)[0]
        trn_idx = np.where(cluster4 != i)[0]
        train_x, val_x = train_eight[:,all_good_features[ii]][trn_idx, :], train_eight[:,all_good_features[ii]][val_idx, :]
        train_y, val_y = y[:,ii][trn_idx], y[:,ii][val_idx]
        clf = LogisticRegression(C=0.2)
        clf.fit(train_x, train_y)
        oof_preds = clf.predict_proba(val_x)[:,1]
        new_oof[val_idx] = oof_preds.reshape(-1,1)
        feature_preds += 0.25*clf.predict_proba(np.hstack(folds[i])[:,all_good_features[ii]])[:,1]
        feature_preds_list.append(clf.predict_proba(np.hstack(folds[i])[:,all_good_features[ii]])[:,1])
            
    oofs.append(new_oof)

    #scores = [f1_score(y[:,ii], (new_oof > th)*1.,) for th in thresholds_0]
    #new_score = np.max(scores)
    #best_scores.append(best_score)
    #best_thresholds.append((np.argmax(scores)+1)/100.)
    
    total_preds.append(feature_preds)
    total_preds_all_folds.append(feature_preds_list)

        

  0%|          | 0/28 [00:00<?, ?it/s]

Fitting feature 0


/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
  4%|▎         | 1/28 [00:01<00:30,  1.13s/it]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433:

Fitting feature 1


/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
  7%|▋         | 2/28 [00:01<00:23,  1.12it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

Fitting feature 24
Fitting feature 25


/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
 93%|█████████▎| 26/28 [00:10<00:00,  2.23it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433

Fitting feature 26


 96%|█████████▋| 27/28 [00:10<00:00,  2.64it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
100%|██████████| 28/28 [00:10<00:00,  3.32it/s]

Fitting feature 27


In [45]:
thresholds_1000 = np.linspace(0.001, 0.999, 999)
best_scores = []
best_thresholds = []
for ii in tqdm(range(28)):
    scores = [f1_score(y[:,ii], (oofs[ii].flatten() > th)*1.,) for th in thresholds_1000]
    best_scores.append(np.max(scores))
    best_thresholds.append((np.argmax(scores)+1)/1000.)

  0%|          | 0/28 [00:00<?, ?it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|██████████| 28/28 [03:21<00:00,  7.08s/it]


In [46]:
np.mean(best_scores)

0.7723700140846949

In [47]:
best_thresholds

[0.574,
 0.563,
 0.492,
 0.498,
 0.518,
 0.476,
 0.441,
 0.469,
 0.016,
 0.017,
 0.012,
 0.276,
 0.409,
 0.387,
 0.608,
 0.032,
 0.242,
 0.247,
 0.2,
 0.343,
 0.416,
 0.36,
 0.351,
 0.511,
 0.228,
 0.431,
 0.325,
 0.018]

In [50]:
oofs_ar = np.array(np.transpose(oofs))
oofs_ar = oofs_ar.reshape((31072, 28))
oof_df = pd.DataFrame(data=oofs_ar)
oof_df['Id'] = cluster4x_folds['Id']
oof_df['cluster4'] = cluster4x_folds['cluster4']

In [51]:
all_preds = []

for i in range(4):
    temp_preds = []
    for j in range(28):
        temp_preds.append(total_preds_all_folds[j][i])
    all_preds.append(pd.DataFrame(data=np.transpose(np.array(temp_preds))))

In [52]:
oof_df.to_csv('../../Human_Protein_Atlas/L2_metafeatures/oof_df_LR_2.csv', index=False)

In [53]:
for i in range(4):
    all_preds[i].to_csv('../../Human_Protein_Atlas/L2_metafeatures/test_preds_LR_2_fold_'+str(i)+'.csv', index=False)